In [53]:
%matplotlib inline

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

plt.style.use('ggplot')


pd.__version__

u'0.17.0'

In [54]:
SSF_subsidies = pd.read_csv("SSF_subsidies.csv")
countries = pd.read_csv("Country_Data.csv")
catch = pd.read_csv("SAU_Catches.csv")
type_EEZ = pd.read_csv("Subtype_Fleet_EEZ.csv")
all_subsidies = pd.read_csv("Subsidies2009.csv")

In [55]:
SSF_subsidies.columns.values

array(['ID', 'Cnumber', 'RegionID', 'RegionName', 'Type', 'SubType',
       'Category', 'ReEst_Subsidy2009', 'NewData', 'Developed', 'HDI_2005',
       'Countries', 'SSFsubsidies_assessed_percent',
       'weighted_by_sectorsize', 'Data_group',
       'SSF_totalCatch_EEZ_percent', '%SSF_totalCatch_Country',
       'SSF_USD_1000', 'Comment', 'DocSource', 'SourceVisitDate', 'Ref',
       'URL', 'User', 'OldComment'], dtype=object)

In [56]:
subsidies_type = pd.merge(left=all_subsidies,right=type_EEZ, how='left')

subsidies_type_countries = pd.merge(left=subsidies_type,right=countries, how='left')

subsidies_type_countries_catch = pd.merge(left=subsidies_type_countries,right=catch, how='left')

catch_countries = pd.merge(left=catch, right=countries, how='left')

catch_by_subregion = catch_countries.groupby(['Subregion']).sum()

catch_by_subregion['mean_catch'] = (catch_by_subregion['SSF_Catch']/catch_by_subregion['Total_Catch'])*100

catch_by_subregion = catch_by_subregion.reset_index()

mean_catch = catch_by_subregion.loc[:,['Subregion','mean_catch']]

subsidies_type_countries_catch_meancatch = pd.merge(left=subsidies_type_countries_catch,right=mean_catch, how='left')

SSF_subsidies_countries = pd.merge(left=SSF_subsidies,right=countries, how='left')

SSF_by_subregion = SSF_subsidies_countries.groupby(['Subregion','Type']).sum()

SSF_by_subregion['mean_subsidies'] = (SSF_by_subregion['SSF_USD_1000']/SSF_by_subregion['ReEst_Subsidy2009'])*100

SSF_by_subregion = SSF_by_subregion.reset_index()

mean_SSFsubsidies = SSF_by_subregion.loc[:,['Subregion','Type','mean_subsidies']]

subsidies_type_countries_catch_meancatch_meanSSFsubsidies = pd.merge(left=subsidies_type_countries_catch_meancatch,right=mean_SSFsubsidies, how='left')

np.sum(subsidies_type_countries_catch_meancatch_meanSSFsubsidies['ReEst_Subsidy2009_original'])


35371111.036526568

In [63]:
subsidies_type_countries_catch_meancatch_meanSSFsubsidies['Factor'] = subsidies_type_countries_catch_meancatch_meanSSFsubsidies['SSF_Catch_percent']/subsidies_type_countries_catch_meancatch_meanSSFsubsidies['mean_catch']

subsidies_type_countries_catch_meancatch_meanSSFsubsidies['SSF_subsidies_estimates'] = subsidies_type_countries_catch_meancatch_meanSSFsubsidies['Factor']*subsidies_type_countries_catch_meancatch_meanSSFsubsidies['mean_subsidies']

SSF_subsidies_short = SSF_subsidies_countries.loc[:,['Countries','Type','SSFsubsidies_assessed_percent','weighted_by_sectorsize','Data_group','SSF_USD_1000']]

SSF_subsidies_estimates_assessed = pd.merge(left=subsidies_type_countries_catch_meancatch_meanSSFsubsidies,right=SSF_subsidies_short,how='left')

SSF_subsidies_estimates_assessed['SSF_subsidies_percent'] = SSF_subsidies_estimates_assessed['SSFsubsidies_assessed_percent'].fillna(SSF_subsidies_estimates_assessed['SSF_subsidies_estimates'])

SSF_subsidies_estimates_assessed['SSF_subsidies_USD'] = (SSF_subsidies_estimates_assessed['ReEst_Subsidy2009_original']/100)*SSF_subsidies_estimates_assessed['SSF_subsidies_percent']

np.sum(SSF_subsidies_estimates_assessed['SSF_subsidies_USD'])


5210024.9500068184

In [64]:
SSF_subsidies_estimates_assessed.to_csv('SSF_subsidies_assessed.csv')